In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi16')

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import pyJHTDB
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
from pyJHTDB import libJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse

from mpiFFT4py.pencil import R2CY

In [3]:
%%px

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']
nu = isotropic1024coarse['nu']

comm = MPI.COMM_WORLD; rank = comm.Get_rank(); nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

nx=Nx//nproc; ny=Ny; nz=Nz; nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx); time = 0.0 

chkSz = 32; slabs = nx//chkSz

[stdout:0] 
n_proc = 16
rank = 0


In [4]:
%%px 

P1 = 4
P2 = 4

In [5]:
%%px 

N = np.array([Nx,Ny,Nz],dtype=int)
L = np.array([Lx,Ly,Lz],dtype=float)

FFT = R2CY(N, L, MPI.COMM_WORLD, "double",P1, communication='Alltoallw')

In [6]:
%%px

if(rank==0):
    print(FFT.real_shape())
    print(FFT.complex_shape())
    print(FFT.real_shape_padded())
    print(FFT.work_shape('3/2-rule'))

[stdout:0] 
(256, 256, 1024)
(256, 1024, 128)
(384, 384, 1536)
(384, 384, 1536)


In [7]:
%%px

print((FFT.comm0.Get_rank(),FFT.comm1.Get_rank()))

[stdout:0] (0, 0)
[stdout:1] (1, 3)
[stdout:2] (1, 0)
[stdout:3] (0, 3)
[stdout:4] (2, 3)
[stdout:5] (0, 1)
[stdout:6] (3, 1)
[stdout:7] (1, 2)
[stdout:8] (2, 2)
[stdout:9] (2, 0)
[stdout:10] (3, 3)
[stdout:11] (1, 1)
[stdout:12] (3, 0)
[stdout:13] (2, 1)
[stdout:14] (0, 2)
[stdout:15] (3, 2)


In [8]:
%%px

p1 = FFT.comm0.Get_rank()
p2 = FFT.comm1.Get_rank()

folder = '/home/idies/workspace/scratch/pencil16'
filename = 'pencil-isotropic1024coarse-('+str(p1)+','+str(p2)+').npz'
file = folder + "/" + filename

vx = np.zeros(FFT.real_shape(), dtype=FFT.float)
vy = np.zeros(FFT.real_shape(), dtype=FFT.float)
vz = np.zeros(FFT.real_shape(), dtype=FFT.float)
    
comm.Barrier(); t1=MPI.Wtime()
content = np.load(file)

vx[:,:,:] = content['u'].astype(FFT.float)
vy[:,:,:] = content['v'].astype(FFT.float)
vz[:,:,:] = content['w'].astype(FFT.float)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Finished loading")
    sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

if(rank==0):
    print("vx shape = "+str(vx.shape))       

[stdout:0] 
Finished loading
Load from disk: 31.69 seconds
vx shape = (256, 256, 1024)


In [9]:
%%px
comm.Barrier(); t1=MPI.Wtime()

K = 0.5*(vx**2+vy**2+vz**2)

slabK = np.sum(K)

avgK=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabK,MPI.DOUBLE],[avgK,MPI.DOUBLE],op=MPI.SUM)
avgK = avgK[0]*(1024**(-3))
avgK = comm.bcast(avgK, root=0)

comm.Barrier(); t2=MPI.Wtime()
if rank==0:
    print("kinectic energy = "+str(avgK))
    print("Computing kinectic energy: ", t2-t1,"s")

[stdout:0] 
kinectic energy = 0.682719227202
Computing kinectic energy:  11.71060299873352 s


In [10]:
%%px
comm.Barrier(); t1=MPI.Wtime()

kx,ky,kz = FFT.get_local_wavenumbermesh()
k2 = kx**2 + ky**2 + kz**2

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Computing wavenumbers:",t2-t1)

[stdout:0] Computing wavenumbers: 3.9223859310150146


In [11]:
%%px 
comm.Barrier(); t1=MPI.Wtime()

cvx = np.zeros(FFT.complex_shape(), dtype=FFT.complex)
cvy = np.zeros(FFT.complex_shape(), dtype=FFT.complex)
cvz = np.zeros(FFT.complex_shape(), dtype=FFT.complex)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Preparing FFT: 0.00 seconds


In [12]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cvx = FFT.fftn(vx,cvx)
cvy = FFT.fftn(vy,cvy)
cvz = FFT.fftn(vz,cvz)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate 3D spatial FFT: 100.64 seconds


In [13]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cA = np.zeros(FFT.complex_shape(), dtype=FFT.complex)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('alocating fourier space derivatives: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] alocating fourier space derivatives: 0.00 seconds


In [14]:
%%px
comm.Barrier(); t1=MPI.Wtime()

A11 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A12 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A13 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A21 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A22 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A23 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A31 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A32 = np.zeros(FFT.real_shape(), dtype=FFT.float)
A33 = np.zeros(FFT.real_shape(), dtype=FFT.float)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocate real space gradients: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Alocate real space gradients: 0.00 seconds


In [15]:
%%px
comm.Barrier(); t1=MPI.Wtime()
 
cA[:,:,:] = np.complex64(0.0+1.0j)*kx[:,:,:]*cvx[:,:,:]
A11 = FFT.ifftn(cA,A11)
A11[:,:,:] = A11[:,:,:]

cA[:,:,:] = np.complex64(0.0+1.0j)*kx[:,:,:]*cvy[:,:,:]
A12 = FFT.ifftn(cA,A12)
A12[:,:,:] = A12[:,:,:]

cA[:,:,:] = np.complex64(0.0+1.0j)*kx[:,:,:]*cvz[:,:,:]
A13 = FFT.ifftn(cA,A13)
A13[:,:,:] = A13[:,:,:]

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate velocity gradient in wavenumber space, and FFT back: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate velocity gradient in wavenumber space, and FFT back: 123.34 seconds


In [16]:
%%px
comm.Barrier(); t1=MPI.Wtime()
 
cA[:,:,:] = np.complex64(0.0+1.0j)*ky[:,:,:]*cvx[:,:,:]
A21 = FFT.ifftn(cA,A21)
A21[:,:,:] = A21[:,:,:]

cA[:,:,:] = np.complex64(0.0+1.0j)*ky[:,:,:]*cvy[:,:,:]
A22 = FFT.ifftn(cA,A22)
A22[:,:,:] = A22[:,:,:]

cA[:,:,:] = np.complex64(0.0+1.0j)*ky[:,:,:]*cvz[:,:,:]
A23 = FFT.ifftn(cA,A23)
A23[:,:,:] = A23[:,:,:]

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate velocity gradient in wavenumber space: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate velocity gradient in wavenumber space: 88.02 seconds


In [17]:
%%px
comm.Barrier(); t1=MPI.Wtime()
 
cA[:,:,:] = np.complex64(0.0+1.0j)*kz[:,:,:]*cvx[:,:,:]
A31 = FFT.ifftn(cA,A31)
A31[:,:,:] = A31[:,:,:]

cA[:,:,:] = np.complex64(0.0+1.0j)*kz[:,:,:]*cvy[:,:,:]
A32 = FFT.ifftn(cA,A32)
A32[:,:,:] = A32[:,:,:]

cA[:,:,:] = np.complex64(0.0+1.0j)*kz[:,:,:]*cvz[:,:,:]
A33 = FFT.ifftn(cA,A33)
A33[:,:,:] = A33[:,:,:]

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate velocity gradient in wavenumber space: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate velocity gradient in wavenumber space: 93.98 seconds


In [18]:
%%px

del cA

In [19]:
%%px

del cvx,cvy,cvz

In [20]:
%%px

del kx,ky,kz

In [21]:
%%px 

A  = [A11,A12,A13,A21,A22,A23,A31,A32,A33]
la = ['A11','A12','A13','A21','A22','A23','A31','A32','A33']

for i in range(9):
    Aav = np.average(A[i])
    slabAavg = np.sum(Aav)
    
    avgA =np.zeros(1,dtype=FFT.float)
    
    comm.Reduce([slabAavg,MPI.DOUBLE],[avgA,MPI.DOUBLE],op=MPI.SUM)
    avgA = avgA[0]/nproc #avgK[0]*(1024**(-3))
    avgA = comm.bcast(avgA, root=0)
    
    if rank==0:
        print("<"+la[i]+"> = "+str(avgA))

[stdout:0] 
<A11> = 5.63785129692e-17
<A12> = -2.01227923213e-16
<A13> = 8.93382590128e-17
<A21> = -1.38777878078e-17
<A22> = 1.82145964978e-17
<A23> = 3.46944695195e-17
<A31> = -1.95231168175e-20
<A32> = -3.33821143905e-20
<A33> = -2.83524426759e-20


In [22]:
%%px 

A  = [A11,A12,A13,A21,A22,A23,A31,A32,A33]
la = ['A11','A12','A13','A21','A22','A23','A31','A32','A33']

for i in range(9):
    A2av = np.average(A[i]**2)
    
    avgA2 =np.zeros(1,dtype=FFT.float)
    
    comm.Reduce([A2av,MPI.DOUBLE],[avgA2,MPI.DOUBLE],op=MPI.SUM)
    avgA2 = avgA2[0]/nproc #avgK[0]*(1024**(-3))
    avgA2 = comm.bcast(avgA2, root=0)
    
    if rank==0:
        print("<"+la[i]+"^2> = "+str(avgA2))

[stdout:0] 
<A11^2> = 33.2611325939
<A12^2> = 66.9627075507
<A13^2> = 67.0288607655
<A21^2> = 65.7716675104
<A22^2> = 33.0683197566
<A23^2> = 65.7397872063
<A31^2> = 66.6281516673
<A32^2> = 66.7571668279
<A33^2> = 33.2177134126


In [40]:
%%px 

A  = [A11,A12,A13,A21,A22,A23,A31,A32,A33]
la = ['A11','A12','A13','A21','A22','A23','A31','A32','A33']

for i in range(9):  
    A2 = A[i]**2
    A2av = np.sum(A2, dtype=FFT.float)
    avgA2 =np.zeros(1,dtype=FFT.float)
    comm.Reduce([A2av,MPI.DOUBLE],[avgA2,MPI.DOUBLE],op=MPI.SUM)
    avgA2 = avgA2[0]/(Nx*Ny*Nz) 
    avgA2 = comm.bcast(avgA2, root=0)
    
    if rank==0:
        print("<"+la[i]+"^2> = "+str(avgA2))
        
    A3 = A[i]/np.sqrt(avgA2)
    A3 = A3**3
    A3av = np.sum(A3, dtype=FFT.float)    
    avgA3 = np.zeros(1,dtype=FFT.float)    
    comm.Reduce([A3av,MPI.DOUBLE],[avgA3,MPI.DOUBLE],op=MPI.SUM)    
    avgA3 = avgA3[0]/(Nx*Ny*Nz) 
    avgA3 = comm.bcast(avgA3, root=0)
    
    skewness = avgA3#/(avgA2**(3/2))
    
    if rank==0:
        print("skewness = <"+la[i]+"^3>/"+"<"+la[i]+"^2>^(3/2) "+"= "+str(skewness) )

[stdout:0] 
<A11^2> = 33.2611325939
skewness = <A11^3>/<A11^2>^(3/2) = -0.590255397263
<A12^2> = 66.9627075507
skewness = <A12^3>/<A12^2>^(3/2) = -0.0262069345797
<A13^2> = 67.0288607655
skewness = <A13^3>/<A13^2>^(3/2) = -0.044600022822
<A21^2> = 65.7716675104
skewness = <A21^3>/<A21^2>^(3/2) = -0.00580459082167
<A22^2> = 33.0683197566
skewness = <A22^3>/<A22^2>^(3/2) = -0.553907679429
<A23^2> = 65.7397872063
skewness = <A23^3>/<A23^2>^(3/2) = -0.0494113131351
<A31^2> = 66.6281516673
skewness = <A31^3>/<A31^2>^(3/2) = 0.00386564467191
<A32^2> = 66.7571668279
skewness = <A32^3>/<A32^2>^(3/2) = 0.00726917637397
<A33^2> = 33.2177134126
skewness = <A33^3>/<A33^2>^(3/2) = -0.576321276815


In [24]:
%%px 

div = A11+A22+A33 

div2av = np.average(div**2)
    
avgdiv2 =np.zeros(1,dtype=FFT.float)
   
comm.Reduce([div2av,MPI.DOUBLE],[avgdiv2,MPI.DOUBLE],op=MPI.SUM)
avgdiv2 = avgdiv2[0]/nproc 
avgdiv2 = comm.bcast(avgdiv2, root=0)

if rank==0:
    print("<(div v)^2> = "+str(avgdiv2))

[stdout:0] <(div v)^2> = 3.28650680116e-09


In [25]:
%%px 

Q = 0.5*( (A12*A21+A23*A32+A13*A31) - (A22*A33+A11*A33+A11*A22) )

In [26]:
%%px

slabQ = np.average(Q)

avgQ=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabQ,MPI.DOUBLE],[avgQ,MPI.DOUBLE],op=MPI.SUM)
avgQ = avgQ[0]/nproc
avgQ = comm.bcast(avgQ, root=0)

if rank==0:
    print("Average Q = "+str(avgQ))

[stdout:0] Average Q = -6.85215773011e-17


In [27]:
%%px

R = -(A11*A22*A33 + A21*A32*A13 + A31*A12*A23 - A13*A22*A31 - A11*A32*A23 - A21*A12*A33)

In [28]:
%%px

slabR = np.average(R)

avgR=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabR,MPI.DOUBLE],[avgR,MPI.DOUBLE],op=MPI.SUM)
avgR = avgR[0]/nproc
avgR = comm.bcast(avgR, root=0)

if rank==0:
    print("Average R = "+str(avgR))

[stdout:0] Average R = 0.00128866982766


In [29]:
%%px

slabQ2 = np.average(Q**2)

avgQ2=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabQ2,MPI.DOUBLE],[avgQ2,MPI.DOUBLE],op=MPI.SUM)
avgQ2 = avgQ2[0]/nproc
avgQ2 = comm.bcast(avgQ2, root=0)

if rank==0:
    print("Average Q^2 = "+str(avgQ2))

[stdout:0] Average Q^2 = 17679.1773945


In [30]:
%%px

slabR = np.average(R)

avgR=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabR,MPI.DOUBLE],[avgR,MPI.DOUBLE],op=MPI.SUM)
avgR = avgR[0]/nproc
avgR = comm.bcast(avgR, root=0)

if rank==0:
    print("<R>/(<Q^2>^3/2) = "+str(avgR/(avgQ2**1.5)))

[stdout:0] <R>/(<Q^2>^3/2) = 5.48212053209e-10


In [31]:
%%px 

del Q,R

In [32]:
%%px

Omega = 0.5*( A12**2 + A21**2 + A13**2 + A31**2 + A23**2 + A32**2 - 2.*(A12*A21+A13*A31+A23*A32) )

In [33]:
%%px

slabO = np.average(Omega)

avgO=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabO,MPI.DOUBLE],[avgO,MPI.DOUBLE],op=MPI.SUM)
avgO = avgO[0]/nproc
avgO = comm.bcast(avgO, root=0)

if rank==0:
    print("Average enstrophy = "+str(avgO))

[stdout:0] Average enstrophy = 249.217753644


In [34]:
%%px

Epsilon  = A11**2 + A22**2 + A33**2 
Epsilon += 0.5*( A12**2 + A21**2 + A23**2 + A32**2 + A13**2 + A31**2 )
Epsilon += A12*A21 + A13*A31 + A23*A32

In [35]:
%%px

slabE = np.average(Epsilon)

avgE=np.zeros(1,dtype=FFT.float)

comm.Reduce([slabE,MPI.DOUBLE],[avgE,MPI.DOUBLE],op=MPI.SUM)
avgE = avgE[0]/nproc
avgE = comm.bcast(avgE, root=0)

if rank==0:
    print("Average strainrate = "+str(avgE))

[stdout:0] Average strainrate = 249.217753647


In [36]:
%%px

if rank==0:
    print("Average strainrate - enstrophy: "+str(avgE-avgO))
    print("Normalized strainrate - enstrophy: "+str((avgE-avgO)/avgO))

[stdout:0] 
Average strainrate - enstrophy: 3.28691385221e-09
Normalized strainrate - enstrophy: 1.31889233578e-11


In [37]:
%%px

del Omega, Epsilon

In [38]:
%%px

eps = 2*nu*avgE

In [39]:
%%px

urms = np.sqrt(2.*avgK/3)
lamb = np.sqrt(15.*nu/eps)*urms
ReTayor = urms*lamb/nu
if rank==0:
    print("nu = "+str(nu))
    print("urms = "+str(urms)+", Reference: 0.6820399")
    print("eps = "+ str(eps))
    print("kinectic energy = "+str(avgK))
    print("Taylor micro-scale ="+str(lamb))
    print("ReLamb = "+str(ReTayor)+", Reference: 426.8378")

[stdout:0] 
nu = 0.000185
urms = 0.674645204139, Reference: 0.6820399
eps = 0.0922105688495
kinectic energy = 0.682719227202
Taylor micro-scale =0.11703522127
ReLamb = 426.795949972, Reference: 426.8378
